In [0]:
# Biblioteca(s)
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, current_timestamp, lit, input_file_name

In [0]:
# Torna disponíveis todas as funções, classes e variáveis definidas em 'lib_utils' no ambiente atual
# Obs: Comando em célula separada, pois não comporta bem com o comando %run

In [0]:
%run "Workspace/programas_python/projetos_e_teste/lib_utils"

In [0]:
# Define schema a ser aplicado no DataFrame
spark_schema = StructType([
    StructField("address_1", StringType(), nullable=True),
    StructField("address_2", StringType(), nullable=True),
    StructField("address_3", StringType(), nullable=True),
    StructField("brewery_type", StringType(), nullable=True),
    StructField("city", StringType(), nullable=True),
    StructField("country", StringType(), nullable=True),
    StructField("id", StringType(), nullable=False), 
    StructField("latitude", StringType(), nullable=True),
    StructField("longitude", StringType(), nullable=True),
    StructField("name", StringType(), nullable=True),
    StructField("phone", StringType(), nullable=True),
    StructField("postal_code", StringType(), nullable=True),
    StructField("state", StringType(), nullable=True),
    StructField("state_province", StringType(), nullable=True),
    StructField("street", StringType(), nullable=True),
    StructField("website_url", StringType(), nullable=True)
])

In [0]:
# Configura os caminhos de armazenamento de dados
db_name_silver = f'db_breweries_silver'
api_path = "api_openbrewerydb"
table_name = "breweries"
bronze_path = "dbfs:/FileStore/bronze_layer"
silver_path = "dbfs:/FileStore/silver_layer"

bronze_source_json = f"{bronze_path}/{api_path}/*.json"
silver_base_path = f"{silver_path}/{api_path}/{db_name_silver}"
silver_sink_delta = f"{silver_base_path}/{table_name}/"


In [0]:
# Cria o banco de dados se ele não existir, usando o nome fornecido em 'db_name_silver'
# O caminho 'silver_sink_delta' especifica o diretório onde o banco de dados será armazenado
create_database_if_not_exists(db_name_silver, silver_sink_delta)

Verificando se o database db_breweries_silver existe. O mesmo será criado em dbfs:/FileStore/shared_uploads/humberto.akita@saomartinho.com.br/silver_layer/api_openbrewerydb/db_breweries_silver/breweries/ caso não exista

In [0]:
# Lê o arquivo JSON do caminho especificado em 'bronze_source_json' usando o Spark,
# aplicando o esquema definido em 'spark_schema'
df_source = spark.read.schema(spark_schema).json(bronze_source_json)

In [0]:
# Cria colunas para possível monitoramento 
df_sink = (df_source.withColumn("dt_silver_entry_utc", current_timestamp())
                    .withColumn("bronze_source_file_name", input_file_name()))

In [0]:
# Escreve o DataFrame 'df_sink' no formato Delta Lake, sobrescrevendo dados existentes
(df_sink.write.format('delta').mode('overwrite')
        # Realiza a partição dos dados com base na coluna 'state'
        .option("mergeSchema", "true").partitionBy("state")
        .saveAsTable(f'{db_name_silver}.{table_name}'))

In [0]:
# Executa a otimização de uma tabela Delta no Spark, para melhorar o desempenho das consultas subsequentes.
spark.sql(f"OPTIMIZE delta.`{silver_sink_delta}`")

Out[25]: DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean>]

In [0]:
# Executa a otimização de uma tabela Delta no Spark, para melhorar o desempenho das consultas subsequentes.
spark.sql(f"VACUUM delta.`{silver_sink_delta}`")

Out[26]: DataFrame[path: string]